## 8 faces fine tuning 
In this nootebook we use a pretrained vgg16 network to extract features of the fc1 layer for the 8 faces dataset. Then we train a fully connected network with those features and our own labels.

In [ ]:
import gzip
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as imgplot
import time
%matplotlib inline
import h5py
from scipy import misc
import os
import random

Loading features for the 8 faces dataset (because of computational reasons we just load them)

In [ ]:
# Downloading the data, if it does not exist
import urllib
import os
if not os.path.isfile('8_faces_EMB.npz'):
    urllib.request.urlretrieve(
    "https://www.dropbox.com/s/3w3x9i7ng7017l3/8_faces_EMB.npz?dl=1",
    "8_faces_EMB.npz")
%ls -hl 8_faces_EMB.npz

In [ ]:
Data=np.load("8_faces_EMB.npz")
print(Data.files)
X_train = Data["arr_0"]
X_valid = Data["arr_1"]
X_test =  Data["arr_2"]
Y_train = Data["arr_3"]
Y_valid = Data["arr_4"]
Y_test =  Data["arr_5"]
print(X_train.shape)
print(X_valid.shape)
print(Y_train.shape)
print(Y_valid.shape)
print(X_test.shape)
print(Y_test.shape)

In [ ]:
def convertToOneHot(vector, num_classes=None):
    result = np.zeros((len(vector), num_classes), dtype='int32')
    result[np.arange(len(vector)), vector] = 1
    return result

In [ ]:
Y_train=convertToOneHot(Y_train,num_classes=8)
Y_valid=convertToOneHot(Y_valid,num_classes=8)

In [ ]:
batch_size = 128
nb_classes = 8
nb_epoch = 30

In [ ]:
import keras
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from keras.models import Sequential


#### Define the network

In [ ]:
model = Sequential()
name = '8_faces_finetune'

#Your code here, built the network



# End of your code

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.summary()


In [ ]:
tensorboard = keras.callbacks.TensorBoard(
        log_dir='tensorboard/8_faces/' + name + '/', 
        write_graph=True,
        histogram_freq=1)

#### Training the network


In [ ]:
history=model.fit(X_train, Y_train, 
                  batch_size=batch_size, 
                  nb_epoch=nb_epoch,
                  verbose=1, 
                  validation_data=(X_valid, Y_valid),
                  callbacks=[tensorboard])

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train acc', 'test acc'], loc='lower right')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train loss', 'test loss'], loc='upper right')
plt.show()

#### Evaluation of the trained network


In [ ]:
preds=np.zeros([len(X_test),8])
for i in range(0,len(X_test)):
    preds[i]=model.predict(X_test[i].reshape(1,4096))

In [ ]:
pred=np.zeros([len(X_test)])
#Your code here, determine the label prediction



# End of your code

In [ ]:
sum(pred==Y_test)/400.


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, pred)

In [ ]:
# Downloading the data, if it does not exist
if not os.path.isfile('Data_test_8_faces_no_cut_100_100.hdf5'):
    urllib.request.urlretrieve(
    "https://www.dropbox.com/s/1oe2cglv2xeb4bg/Data_test_8_faces_no_cut_100_100.hdf5?dl=1",
    "Data_test_8_faces_no_cut_100_100.hdf5")
%ls -hl Data_test_8_faces_no_cut_100_100.hdf5

In [ ]:
h5f_X = h5py.File('Data_test_8_faces_no_cut_100_100.hdf5', 'r')
print(list(h5f_X.keys()))
X_test = h5f_X['X_test_8_faces']
print(X_test.shape)
Y_test = h5f_X['Y_test_8_faces']
print(Y_test.shape)


#### Visualize right and wrong classified images

In [ ]:
Klasse=1
right = np.where(Y_test[:]==Klasse)[0][np.where(np.in1d(np.where(Y_test[:]==Klasse),np.where(pred[:]==Klasse)))]
wrong = np.where(Y_test[:]==Klasse)[0][(np.in1d(np.where(Y_test[:]==Klasse),np.where(pred[:]==Klasse)))==False]

plt.figure(figsize=(18,18))
for i in range(0,len(right)):
    plt.subplot(10,10,(i+1))
    plt.imshow(np.asarray(X_test[right[i]],dtype="uint8"))
    plt.axis('off')
    plt.title('right')
    
plt.figure(figsize=(18,18))
for i in range(0,len(wrong)):
    plt.subplot(10,10,(i+1))
    plt.imshow(np.asarray(X_test[wrong[i]],dtype="uint8"))
    plt.axis('off')
    plt.title('wrong')